# Integration of Keras Models with Tensorflow Estimator
In this tutorial, we want to integrate Keras models with [Tensorflow Estimators](https://www.tensorflow.org/extend/estimators). Estimator is a powerful feature that brings flexibility to run an experiment and helps developers to avoid boilerplate code for training/testing a model. Check this [interesting video](https://www.youtube.com/watch?v=t64ortpgS-E&list=PLOU2XLYxmsIKGc_NBoIhTn2Qhraji53cv&index=6) for an introduction to Estimator. Keras provides an abstract api for developing neural networks and helps developer for fast prototyping. By reading this tutorial, you will be able to use Keras models in Tensorflow ecosystem.



# Prerequisit
* [Install Tensorflow 1.1](https://www.tensorflow.org/versions/r1.1/install/) or higher. 
* Basic understanding of [Functional API of Keras](https://keras.io/getting-started/functional-api-guide/).
* Basic understanding of [Tensorflow Estimators](https://www.tensorflow.org/versions/r0.12/tutorials/estimators/).

If you only need Keras as an interface for Tensorlow, please follow [this tutorial](https://blog.keras.io/keras-as-a-simplified-interface-to-tensorflow-tutorial.html). 


# Building a Keras model for An Abalone Age Predictor

This tutorial essentially follows the same approach described in [this tutorial](https://www.tensorflow.org/extend/estimators), however, instead of using Tensorflow layers, we want to use Keras layers.

## Building an estimator for a Keras Model
To build an estimator, we need to implement [model_fn function](https://www.tensorflow.org/versions/r0.12/tutorials/estimators/#constructing_the_model_fn). The model_fn function is invoked in a different context (TRAIN, INFER and EVAL) to set up the estimator. This function essentially is a function that returns [ModelFnOps](https://www.tensorflow.org/api_docs/python/tf/contrib/learn/ModelFnOps) which encapsulates the following variables: 
1. **Lost Tensor** (required for *EVAL* and *TRAIN* modes): This tensor indicates how closely the model's predictions match the target values.
2. **Train Operation** (required for *TRAIN* mode): This operation will be used to train a model, which is typically an optimization operation on a loss function. For our purpose, we use the SGD algorithm on the mean square error.
3. **Prediction Tensors** (required for *EVAL* and *INFER* modes):  A dict that maps key names of your choice to Tensors containing the predictions from the model. For our example, we return a dictionary that only contains the predicted age.
4. **Evaluate MetricSpec** (required for *EVAL* mode): This is a dictionary of [MetricSpec](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/learn/python/learn/metric_spec.py). *MetricSpec*s uses a *prediction_key* to associate a [metric](https://www.tensorflow.org/versions/r0.12/api_docs/python/contrib.metrics/#metric-ops) to one of the predictions returned by *Prediction Tensors* which has the same key in the Prediction Tensors dictionary. If there is only one prediction for a model (like the model used in this tutorial), there is no need to set *prediction_key*. 

To build these operations, *model_fn* is fit with four parameters:
1. **features**: This is an object that contains instances of a dataset. In our example, we use a numpy matrix that stores our features.
2. **targets**: A tensor containing the gold-standard labels.
3. **mode**: The context that model_fn was called. This context can be TRAIN, EVAL and INFER mode.
4. **params**: Custome parameters for the model. For our example, we set the learning rate of the optimizer from params.

Now we covered the model_fn function, let implement it for our model. We would like to predicate the age of an [abalone](https://en.wikipedia.org/wiki/Abalone). To do this, our model consists of two fully connected layers with the relu activation function followed by a linear layer without non-linearity that return the predicted age.

In [1]:
from tensorflow.contrib.keras.python.keras.layers import Input, Dense
from tensorflow.contrib.keras.python.keras.models import Model
from tensorflow.contrib.keras.python.keras.optimizers import SGD
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib
import tensorflow as tf
import numpy as np
import os.path
import urllib

model = None
i = 0

def build_model():
  global i
  i = i + 1
  inputs = Input(shape=(7,))
  # Note that features is a Tensorflow tensor and Dense is a Keras layer.

#   predictions = tf.reshape(output_layer, [-1])
  return Model(inputs=inputs, outputs=output_layer)

def model_fn(features, targets, mode, params):
  """Model function for Estimator.
   # Logic to do the following:
   # 1. Configure the model via Keras functional api
   # 2. Define the loss function for training/evaluation using Tensorflow.
   # 3. Define the training operation/optimizer using Tensorflow operation/optimizer.
   # 4. Generate predictions as Tensorflow tensors.
   # 5. Generate necessary evaluation metrics.
   # 6. Return predictions/loss/train_op/eval_metric_ops in ModelFnOps object"""
  
  print("############### %s ###################" % mode)
  # 1. Configure the model via Keras functional api

  first_hidden_layer = Dense(10, activation='relu', name = str(i))(features)
  second_hidden_layer = Dense(10, activation='relu')(first_hidden_layer)
  output_layer = Dense(1, activation='linear')(second_hidden_layer)  

  predictions = tf.reshape(output_layer, [-1])
    
  # 2. Define the loss function for training/evaluation using Tensorflow.
  loss = tf.losses.mean_squared_error(targets, predictions)

  # 3. Define the training operation/optimizer using Tensorflow operation/optimizer.
  train_op = tf.contrib.layers.optimize_loss(
    loss=loss,
    global_step=tf.contrib.framework.get_global_step(),
    learning_rate=params["learning_rate"],
    optimizer="SGD")

  # 4. Generate predictions as Tensorflow tensors.
  predictions_dict = {"ages": predictions}

  # 5. Generate necessary evaluation metrics.
  # Calculate root mean squared error as additional eval metric
  eval_metric_ops = {
      "rmse": tf.metrics.root_mean_squared_error(
          tf.cast(targets, tf.float32), predictions)
  }

  # 6. Return predictions/loss/train_op/eval_metric_ops in ModelFnOps object
  return model_fn_lib.ModelFnOps(
      mode=mode,
      predictions=predictions_dict,
      loss=loss,
      train_op=train_op,
      eval_metric_ops=eval_metric_ops)



Now lets download the dataset and try our model:

In [2]:
import tensorflow as tf
import numpy as np
import os.path
import urllib

TRAIN_DATASET_URL="http://download.tensorflow.org/data/abalone_train.csv"
TEST_DATASET_URL="http://download.tensorflow.org/data/abalone_test.csv"
PREDICT_DATASET_URL="http://download.tensorflow.org/data/abalone_predict.csv"

def get_dataset(url, filename):
  if not os.path.exists(filename):
    urllib.request.urlretrieve(
        url,
        filename)
  dataset = tf.contrib.learn.datasets.base.load_csv_without_header(
    filename=filename, target_dtype=np.int, features_dtype=np.float32)
  print("Dataset has been downloaded in %s, its size is %s" % (filename, dataset.data.shape))
  return dataset

training_set = get_dataset(TRAIN_DATASET_URL, "training.csv")
test_set = get_dataset(TEST_DATASET_URL, "test.csv")
predict_set = get_dataset(PREDICT_DATASET_URL, "predict.csv")

Dataset has been downloaded in training.csv, its size is (3320, 7)
Dataset has been downloaded in test.csv, its size is (850, 7)
Dataset has been downloaded in predict.csv, its size is (7, 7)


To train the model we need to instantiate an Estimator: 

In [3]:
from tensorflow.contrib.learn.python.learn.estimators import estimator
LEARNING_RATE = 0.001
# Set model params
model_params = {"learning_rate": LEARNING_RATE}

# Instantiate Estimator
est = estimator.Estimator(model_fn=model_fn, params=model_params)

est.fit(x=training_set.data, y=training_set.target, steps=5000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_task_type': None, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_model_dir': None, '_save_checkpoints_steps': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_master': '', '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_is_chief': True, '_environment': 'local', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1126a24e0>, '_num_worker_replicas': 0, '_evaluation_master': ''}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving int

/Users/majid/anaconda/envs/tensorflow/lib/python3.5/site-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/_g/9zy1zpy55c59kbl62wkkj6hm0000gn/T/tmpsilesg20/model.ckpt.
INFO:tensorflow:loss = 100.77, step = 1
INFO:tensorflow:global_step/sec: 316.869
INFO:tensorflow:loss = 7.55002, step = 101 (0.317 sec)
INFO:tensorflow:global_step/sec: 301.2
INFO:tensorflow:loss = 7.26714, step = 201 (0.332 sec)
INFO:tensorflow:global_step/sec: 276.011
INFO:tensorflow:loss = 7.11362, step = 301 (0.362 sec)
INFO:tensorflow:global_step/sec: 317.903
INFO:tensorflow:loss = 7.01887, step = 401 (0.315 sec)
INFO:tensorflow:global_step/sec: 294.88
INFO:tensorflow:loss = 6.9508, step = 501 (0.339 sec)
INFO:tensorflow:global_step/sec: 314.667
INFO:tensorflow:loss = 6.89405, step = 601 (0.318 sec)
INFO:tensorflow:global_step/sec: 319.556
INFO:tensorflow:loss = 6.84138, step = 701 (0.313 sec)
INFO:tensorflow:global_step/sec: 327.944
INFO:tensorflow:loss = 6.78933, step = 801 (0.305 sec)
INFO:tensorflow:global_step/sec: 

Estimator(params={'learning_rate': 0.001})

Finally, we evaluate our model on the test-dataset and predict the age of abalone in the predict-dataset.

In [4]:

# Score accuracy
ev = est.evaluate(x=test_set.data, y=test_set.target, steps=1)
print("Loss: %s" % ev["loss"])
print("Root Mean Squared Error: %s" % ev["rmse"])

# Print out predictions
predict_results = est.predict(x=predict_set.data)
i = 0
for p in predict_results:
  i = i + 1
  print("Prediction %s: %s" % (i + 1, p))

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
############### eval ###################


/Users/majid/anaconda/envs/tensorflow/lib/python3.5/site-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Starting evaluation at 2017-04-12-21:56:52
INFO:tensorflow:Restoring parameters from /var/folders/_g/9zy1zpy55c59kbl62wkkj6hm0000gn/T/tmpsilesg20/model.ckpt-5000
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-04-12-21:56:53
INFO:tensorflow:Saving dict for global step 5000: global_step = 5000, loss = 5.37203, rmse = 2.31776
Loss: 5.37203
Root Mean Squared Error: 2.31776
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
############### infer ###################
INFO:tensorflow:Restoring parameters from /var/folders/_g/9zy1zpy55c59kbl62wkkj6hm0000gn/T/tmpsilesg20/model.ckpt-5000
Prediction 2: {'ages': 4.4740086}
Prediction 3: {'ages': 10.712712}
Prediction 4: {'ages': 7.1116824}
Prediction 5: {